In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

import os
from pathlib import Path

In [2]:
DATA_DIR = Path("../../../data/")
DATA_DIR.mkdir(parents=True, exist_ok=True)

LATEX_OUT = Path("../../../docs/latex_utils/tables")
LATEX_OUT.mkdir(parents=True, exist_ok=True)

In [3]:
def save_plot(
    plot: plt.Figure,
    filename: str,
    format: str = "png",
    dpi: int = 300,
    close: bool = True,
):
    PLOTS_DIR.mkdir(parents=True, exist_ok=True)
    filepath = PLOTS_DIR / f"{filename}.{format}"
    try:
        plot.savefig(filepath, format=format, dpi=dpi, bbox_inches="tight")
        if close:
            plt.close(plot)
        print(
            f"\nPlot {filename}.{format} saved correctly in {PLOTS_DIR}/{filename}.{format}"
        )
    except Exception as e:
        print(f"\nCould not save plot {filename}.{format}. Reason: {e}")

In [4]:
def save_latex_table(df, filename: str, rename_map: dict, caption: str, label: str):
    try:
        table_tex = df.rename(columns=rename_map).to_latex(
            index=False,
            float_format="%.4f",
            caption=caption,
            label=label,
        )
        with open(LATEX_OUT / filename, "w") as f:
            f.write(table_tex)
        print(f"\nFile {filename} exported correctly in {LATEX_OUT}/{filename}")
    except Exception as e:
        print(f"\nCould not export {filename}. Reason: {e}")

In [7]:
years = list(range(1962, 1982))
Y = [
    51.1,
    52.3,
    53.6,
    49.6,
    56.8,
    70.1,
    80.5,
    81.2,
    80.3,
    77.7,
    78.3,
    74.5,
    77.8,
    85.6,
    89.4,
    97.5,
    105.2,
    117.7,
    135.9,
    162.1,
]
X2 = [
    560.3,
    590.5,
    632.4,
    684.9,
    749.9,
    793.0,
    865.0,
    931.4,
    992.7,
    1077.6,
    1185.9,
    1326.4,
    1434.2,
    1549.2,
    1748.0,
    1918.3,
    2163.9,
    2417.8,
    2633.1,
    2937.7,
]
X3 = [
    0.6,
    0.9,
    1.1,
    1.4,
    1.6,
    1.0,
    0.8,
    1.5,
    1.0,
    1.5,
    2.95,
    4.8,
    10.3,
    16.0,
    14.7,
    8.3,
    11.0,
    13.0,
    15.3,
    18.0,
]
X4 = [
    16.0,
    16.4,
    16.7,
    17.0,
    20.2,
    23.1,
    25.6,
    24.6,
    24.8,
    27.1,
    21.5,
    24.3,
    26.8,
    29.5,
    30.4,
    33.3,
    38.0,
    46.2,
    57.6,
    68.9,
]
X5 = [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

df = pd.DataFrame({"year": years, "Y": Y, "X2": X2, "X3": X3, "X4": X4, "X5": X5})
df.head(30)

,year,Y,X2,X3,X4,X5
0,1962,51.1,560.3,0.60,16.0,0
1,1963,52.3,590.5,0.90,16.4,0
2,1964,53.6,632.4,1.10,16.7,0
3,1965,49.6,684.9,1.40,17.0,1
4,1966,56.8,749.9,1.60,20.2,1
5,1967,70.1,793.0,1.00,23.1,1
6,1968,80.5,865.0,0.80,25.6,1
7,1969,81.2,931.4,1.50,24.6,1
8,1970,80.3,992.7,1.00,24.8,1
9,1971,77.7,1077.6,1.50,27.1,1


In [ ]:
X = sm.add_constant(df[["X2", "X3", "X4", "X5"]])
ols = sm.OLS(df["Y"], X).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.978
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                     169.5
Date:                Tue, 09 Sep 2025   Prob (F-statistic):           2.73e-12
Time:                        00:36:14   Log-Likelihood:                -56.865
No. Observations:                  20   AIC:                             123.7
Df Residuals:                      15   BIC:                             128.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         19.7122      3.351      5.883      0.0

In [9]:
b = ols.params
se = ols.bse
print("\n--- Coeficientes centrales ---")
for name in ["const", "X2", "X3", "X4", "X5"]:
    print(f"{name:>6}: {b[name]:10.4f} (se = {se[name]:.4f})")

print(f"\nR2 = {ols.rsquared:.4f} | R2 ajustado = {ols.rsquared_adj:.4f}")


--- Coeficientes centrales ---
 const:    19.7122 (se = 3.3509)
    X2:     0.0164 (se = 0.0065)
    X3:    -0.2261 (se = 0.4556)
    X4:     1.3967 (se = 0.2608)
    X5:     5.3564 (se = 3.0201)

R2 = 0.9784 | R2 ajustado = 0.9726


In [11]:
print("\n--- (b) Signos esperados vs estimados ---")
esperados = {"X2": "pos", "X3": "pos", "X4": "pos", "X5": "pos"}


def cumple_signo(valor, esperado):
    return (valor > 0) if esperado == "pos" else (valor < 0)


for var, sgn in esperados.items():
    ok = cumple_signo(b[var], sgn)
    print(f"{var}: estimado = {b[var]:.4f} | esperado {sgn} -> ¿cumple? {ok}")

efecto_dummy = b["X5"]
print(
    f"\nLa dummy X5 (conflictos ≥100k) aumenta Y en {efecto_dummy:.2f} miles de millones (ceteris paribus)."
)


--- (b) Signos esperados vs estimados ---
X2: estimado = 0.0164 | esperado pos -> ¿cumple? True
X3: estimado = -0.2261 | esperado pos -> ¿cumple? False
X4: estimado = 1.3967 | esperado pos -> ¿cumple? True
X5: estimado = 5.3564 | esperado pos -> ¿cumple? True

La dummy X5 (conflictos ≥100k) aumenta Y en 5.36 miles de millones (ceteris paribus).


In [12]:
try:
    from statsmodels.stats.outliers_influence import variance_inflation_factor

    X_no_const = df[["X2", "X3", "X4", "X5"]].astype(float).values
    vif = [variance_inflation_factor(X_no_const, i) for i in range(X_no_const.shape[1])]
    print("\n--- VIF (multicolinealidad; >10 es alto) ---")
    for name, v in zip(["X2", "X3", "X4", "X5"], vif):
        print(f"{name}: VIF = {v:.2f}")
except Exception as e:
    print("No se pudo calcular VIF:", e)


--- VIF (multicolinealidad; >10 es alto) ---
X2: VIF = 80.20
X3: VIF = 13.28
X4: VIF = 61.70
X5: VIF = 2.49


In [13]:
def signo(x):
    return "+" if x > 0 else "-"


txt = (
    f"\nResumen rápido:\n"
    f" • b2 (PNB)  = {b['X2']:.4f} ({signo(b['X2'])}) — esperado positivo.\n"
    f" • b3 (Mil)  = {b['X3']:.4f} ({signo(b['X3'])}) — esperado positivo.\n"
    f" • b4 (Aero) = {b['X4']:.4f} ({signo(b['X4'])}) — esperado positivo.\n"
    f" • b5 (Dummy conflictos) = {b['X5']:.4f} ({signo(b['X5'])}).\n"
    f"R2_aj = {ols.rsquared_adj:.3f}. "
    f"Conclusión: signos y magnitudes comparables con expectativas a priori "
    f"(crecimiento económico, actividad militar/aeroespacial y guerras elevan el presupuesto)."
)
print(txt)


Resumen rápido:
 • b2 (PNB)  = 0.0164 (+) — esperado positivo.
 • b3 (Mil)  = -0.2261 (-) — esperado positivo.
 • b4 (Aero) = 1.3967 (+) — esperado positivo.
 • b5 (Dummy conflictos) = 5.3564 (+).
R2_aj = 0.973. Conclusión: signos y magnitudes comparables con expectativas a priori (crecimiento económico, actividad militar/aeroespacial y guerras elevan el presupuesto).
